In [3]:
import numpy as np

transcript_length = 60
available_length = 100


Apparently, the number of paths of length $\ell$ from node $m$ to $n$ can be
computed by using the adjacency matrix $A$ of the graph:

$$ N_\ell (m, n) = A^\ell_{m, n} $$

Using the CTC output format, the adjacency matrix of the graph representing
character transitions are upper triangular matrices roughly of the form

$$ \begin{pmatrix} 1 & 1 & 0 & 0 \\ 0 & 1 & 1 & 1\\ 0 & 0 & 1 & 1 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

(except in those cases where there is a blank -- there is no skip transition
to the second next character then).

In [13]:
def get_adjacency_matrix(seqlen: int):
    mat = np.ones((2 * seqlen + 1, 2 * seqlen + 1))
    mat = np.triu(mat)
    mat = np.tril(mat, 3)

    diag = np.arange(0, mat.shape[0] - 2, 2)
    mat[diag, diag+2] = 0

    return mat

A = get_adjacency_matrix(transcript_length)

In [34]:
from numpy import linalg as la


la.matrix_power(A, available_length)[1, -2]

5.204013246670726e+52

In [38]:
k = available_length - transcript_length
t = available_length
(3 ** (k + 2) - 2 ** (k + 3)) * t

10941898033541933700100

In [35]:
def get_simp_adjacency_matrix(seqlen: int):
    mat = np.ones((seqlen, seqlen))
    mat = np.triu(mat)
    mat = np.tril(mat, 2)

    return mat
B = get_simp_adjacency_matrix(transcript_length)
la.matrix_power(B, available_length)[0, -1]

6.073973740858229e+40

In [53]:
def random_ctc_output(seqlen: int, nclasses: int, batch: int=1):
    A = np.random.rand(batch, seqlen, nclasses)
    expA = np.exp(A)
    A = expA / expA.sum(axis=-1)[:,:,None]
    return A

def random_ctc_gt(seqlen: int, nclasses: int, batch: int=1):
    Y = np.random.randint(0, nclasses + 1, (batch, seqlen))
    lengths = np.random.normal(seqlen / 2, seqlen / 10, (batch,)).astype(int)

    return Y, lengths


ctc_out = random_ctc_output(100, 50, 1)

ctc_gt, ctc_gt_lengths = random_ctc_gt(60, 50, 1)
ctc_gt[0,:ctc_gt_lengths[0]]

array([35, 30, 36, 46, 18,  2, 19, 34,  2, 23, 43, 49, 21, 38, 40, 30, 29,
       23,  7,  6,  6, 25, 23,  2, 45, 36, 13,  1,  1,  2, 11, 30,  9, 30,
       40, 44])

In [59]:
from numpy.typing import ArrayLike
from typing import List

class Beam:
    def __init__(
            self,
            out_index: int,
            dec_indices: List,
            score: float,
    ) -> None:
        self._out_index: int = out_index
        self._dec_indices: List = dec_indices
        self._score: float = score

    def get_score(
            self,
    ) -> float:
        return self._score


def expand_beam(
        beam: Beam,
        probs: ArrayLike,
        transcript: ArrayLike
    ) -> List[Beam]:
    output = []

    output.append(Beam(
        beam._out_index,
        beam._dec_indices + [0],
        beam._score + np.log(probs[0]),
    ))

    output.append(Beam(
        beam._out_index,
        beam._dec_indices + [transcript[beam._out_index]],
        beam._score + np.log(probs[transcript[beam._out_index]]
    ))

# def decode(matrix: ArrayLike, target: ArrayLike) -> ArrayLike:
#     for column in matrix.T:
#         ...
#     return matrix

matrix = ctc_out[0]
target = ctc_gt[0,:ctc_gt_lengths[0]]

beams = [
    Beam(0, [0], np.log(matrix[0, 0])),
    Beam(0, [0], np.log(matrix[0, target[0]]))
]

beams.sort(key=Beam.get_score, reverse=True)


for column in matrix.T[1:]:
    ...
